In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\recommendation-engine'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    embedding_dim: int
    epochs: int
    learning_rate: float
    alpha: float
    k: int 

In [5]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.HybridRecommender

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            model_name=config.model_name,
            embedding_dim=params.embedding_dim,
            epochs=params.epochs,
            learning_rate=params.learning_rate,
            alpha=params.alpha,
            k=params.k
        )

In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.sparse import csr_matrix
import joblib
from src.hybrid_recommender import logger

In [ ]:
class RecommendationEngine:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.user_encoder = None
        self.item_encoder = None
        self.organizer_encoder = None
        self.user_decoder = None
        self.item_decoder = None
        self.organizer_decoder = None
        
    def load_models(self):
        """Load trained models and encoders"""
        
        models_dir = self.config.root_dir
        self.implicit_model = joblib.load(os.path.join(models_dir, 'implicit_model.joblib'))
        self.nn_model = tf.keras.models.load_model(os.path.join(models_dir, 'nn_model.h5'))
        self.scaler = joblib.load(os.path.join(models_dir, 'scaler.joblib'))
        encoders = joblib.load(os.path.join(models_dir, 'encoders.joblib'))
        
        self.user_encoder = encoders['user_encoder']
        self.item_encoder = encoders['item_encoder']
        self.organizer_encoder = encoders['organizer_encoder']
        self.user_decoder = {i: u for u, i in self.user_encoder.items()}
        self.item_decoder = {i: m for m, i in self.item_encoder.items()}
        self.organizer_decoder = {i: o for o, i in self.organizer_encoder.items()}
        
    def recommend(self, user_id, n_recommendations=10):
        """Generate recommendations for a user"""

        user_encoded = self.user_encoder.get(user_id)
        if user_encoded is None:
            return []  # Return empty list for cold-start users
        
        user_items = csr_matrix(([1], ([0], [0])), 
                          shape=(1, len(self.item_encoder)))
        implicit_recs = self.implicit_model.recommend(
            user_encoded, 
            user_items,
            N=n_recommendations*3  # Get more candidates for NN to score
        )
        
        recommended_events = [self.item_decoder[item] for item in implicit_recs[0]]
        
        df = pd.read_csv(self.config.train_data_path)
        event_organizer_map = df.drop_duplicates('event_id').set_index('event_id')['organizer_id'].to_dict()
        
        organizer_ids = [event_organizer_map.get(event_id, 0) for event_id in recommended_events]  # default to 0 if not found
        organizer_encoded = [self.organizer_encoder.get(org_id, 0) for org_id in organizer_ids]  # default to 0 if not found
        
        user_array = np.array([user_encoded] * len(implicit_recs[0]))
        event_array = np.array(implicit_recs[0])
        organizer_array = np.array(organizer_encoded)
        
        nn_scores = self.nn_model.predict([user_array, event_array, organizer_array])
        nn_scores = self.scaler.inverse_transform(nn_scores.reshape(-1, 1)).flatten()
        
        combined_scores = implicit_recs[1] * 0.6 + nn_scores * 0.4
        top_indices = np.argsort(combined_scores)[::-1][:n_recommendations]
        
        recommendations = []
        for idx in top_indices:
            event_id = self.item_decoder[implicit_recs[0][idx]]
            score = combined_scores[idx]
            organizer_id = organizer_ids[idx]
            recommendations.append({
                            'event_id': event_id,
                            'organizer_id': organizer_id,
                            'score': score
                        })        
        return recommendations

In [9]:

try:
    config = ConfigurationManager()
    recommender_config = config.get_model_trainer_config()
    recommender = RecommendationEngine(config=recommender_config)
    recommender.load_models()
    recommendations = recommender.recommend(user_id='f3b0e947-0637-4a57-bae5-2f0f12257bcc', n_recommendations=10)
    print(recommendations)
except Exception as e:
    logger.exception("Error in testing hybrid recommender")
    raise e


[2025-07-01 16:02:51,183: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-01 16:02:51,195: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-01 16:02:51,198: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-01 16:02:51,200: INFO: common: created directory at: artifacts]
[2025-07-01 16:02:51,201: INFO: common: created directory at: artifacts/model_trainer]
[2025-07-01 16:02:51,375: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


d:\newst\miniconda3\envs\recommendation-engine-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
[{'event_id': '7bdb96e4-a679-48f4-aa28-4acc33b3c789', 'organizer_id': '65384a4c-d3bf-4dc9-839c-93a7c2f2621e', 'score': np.float32(0.94807005)}, {'event_id': '58da7fe0-af1a-4fa3-b802-d86fc4878a6f', 'organizer_id': '0ff6a69d-4711-4ef7-9574-cea46ebe21ec', 'score': np.float32(0.89296025)}, {'event_id': 'ce9e2f1a-111b-44b4-8a27-fce84a4a77d2', 'organizer_id': '46653989-842a-4c23-a5b0-1a5cdb79843f', 'score': np.float32(0.6521462)}, {'event_id': 'ae8c7a8f-d799-47a4-bcf7-9b27681f95c6', 'organizer_id': 'a844236c-7e3d-4f9f-9b78-9860361d3f8f', 'score': np.float32(0.6287296)}, {'event_id': '3c24304c-da0e-4a24-a9e1-21f6f168d541', 'organizer_id': '1aafbaa0-9efd-4d51-adbb-310e8a3355d5', 'score': np.float32(0.5647768)}, {'event_id': '3ed3aca0-41dd-45aa-811d-d6c3fe3b7060', 'organizer_id': '32a7a562-245c-46a8-8974-130a84b087c1', 'score': np.float32(0.54017246)}, {'event_id': 'cf28c1eb-c7f7-4688-a078-b198542500c6', 'organizer_id': '79920c5e-ea88-4068-979d-0957711ffa8